In [31]:
import requests
import pandas as pd
import json
from pprint import pprint
import time
import gspread

Set up two urls - one for cycle to date totals, one for q3

In [9]:
url_CTD = "https://api.open.fec.gov/v1/committee/idgoeshere/totals/?page=1&sort_hide_null=false&sort=-cycle&sort_null_only=false&per_page=20&sort_nulls_last=false&cycle=2022&api_key=TvP83x8cUz6L2MAHLXPRCPchTzuQLHfU7Pel9pfv"

Filled out from API: filings —> committee —> committee id —> filings

In [25]:
url_q3 = "https://api.open.fec.gov/v1/committee/idgoeshere/filings/?api_key=TvP83x8cUz6L2MAHLXPRCPchTzuQLHfU7Pel9pfv&sort=-receipt_date&sort_nulls_last=false&report_year=2022&per_page=20&page=1&sort_hide_null=false&sort_null_only=false&cycle=2022&report_type=Q3"

In [11]:
api_key = "TvP83x8cUz6L2MAHLXPRCPchTzuQLHfU7Pel9pfv"

List IDs of candidates we want to pull data for

In [12]:
senate_ids = [["C00696526","Mark Kelly"],["C00778407","Jim Lamon"],["C00784165", "Blake Masters"],["C00781898","Mark Brnovich"],["C00780791","Mike McGuire"],["C00736876","Raphael Warnock"],["C00787853","Hershel Walker 1"],["C00772962","Hershel Walker 2"],["C00776328","Latham Saddler"],["C00787135","Adam Laxalt 1"],["C00772897","Adam Laxalt 2"],["C00783936","Sam Brown"],["C00765800","John Fetterman"],["C00800623","Dave McCormick"],["C00795930","Mehmet Oz 1"],["C00772871","Mehmet Oz 2"],["C00657411","Conor Lamb"],["C00769604", "Alex Lasry"],["C00776740", "Sarah Godlewski"],["C00784959","Mandela Barnes"],["C00800847","Chuck Morse"],["C00800326","Kevin Smith"],["C00777904", "Cheri Beasley"],["C00614776","Ted Budd 1"],["C00772889","Ted Budd 2"],["C00776757","Patrick McCrory"],["C00790550","Marjorie Eastman"],["C00575548","Catherine Cortez Masto"],["C00482984","Ron Johnson"],["C00588772","Maggie Hassan"],["C00543231","Mark Walker"]]

Create a loop, will go through every ID in the senate_id list

Use RESP replace to exchange the "idgoeshere" holder w/ actual ID

Set variables, using json, for each layer of data we want to grab -- find where the breakdown of list/dict we were looking at to get this before?

In [38]:
data_list = []
for senator in senate_ids:

# For cycle to date
# THIS WORKS

    d = {}
    resp = requests.get(url_CTD.replace("idgoeshere",senator[0]))
#     print(resp.text)
    json_resp = json.loads(resp.text)
    receipts_total = json_resp["results"][0]["receipts"]
    d['total_receipts'] = receipts_total
    d['senate_id'] = senator[0]
    d['candidate_name'] = senator[1]
    
# For Q3
# THIS DOESN'T WORK -- getting "list index out of range"

    resp_q3 = requests.get(url_q3.replace("idgoeshere",senator[0]))
#     print(resp_q3.text)
    json_resp_q3 = json.loads(resp_q3.text)
    q3_results = json_resp_q3["results"]

    d['q3_filing'] = q3_results[0]['total_receipts']
    
    data_list.append(d)

IndexError: list index out of range

Putting that data in a DF

In [21]:
senate_df = pd.DataFrame(data_list)

In [22]:
senate_df

,total_receipts,senate_id,candidate_name,q3_filing
0,54141014.69,C00696526,Mark Kelly,13597355.33
1,15077976.55,C00778407,Jim Lamon,1216070.96
2,4950182.20,C00784165,Blake Masters,827574.06
3,3118529.11,C00781898,Mark Brnovich,526532.20
4,2467896.25,C00780791,Mike McGuire,348256.77
5,84914584.33,C00736876,Raphael Warnock,11691245.54
6,20105065.16,C00787853,Hershel Walker 1,3648123.55
7,11065.40,C00772962,Hershel Walker 2,112025.13
8,4263891.70,C00776328,Latham Saddler,172353.33
9,7239536.54,C00787135,Adam Laxalt 1,1351815.17


In [188]:
senate_df.to_csv("2022q2-senate.csv",index=False)

Followed this tutorial: https://docs.gspread.org/en/latest/oauth2.html
Gave access to Drive + Sheets API, named sheet, gave access to client-email from json

In [32]:
gc = gspread.service_account(filename='/Users/achoi/Downloads/senate-campfin-q3.json')

In [33]:
sh = gc.open("2022-q3-senate")

In [37]:
worksheet = sh.worksheet("Sheet1")
worksheet.update([senate_df.columns.values.tolist()] + senate_df.values.tolist())

{'spreadsheetId': '1LVQ4PD3-i6z2ozGzXv_gkuOmUtnyu2YQ5LzrZ6HwC2A',
 'updatedRange': 'Sheet1!A1:D32',
 'updatedRows': 32,
 'updatedColumns': 4,
 'updatedCells': 128}